In [1]:
import pandas as pd

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('mode.chained_assignment',  None)

get_in_off_merged_data = pd.read_csv('./data/get_in_off_bus.csv')

# getrouteinfoall
getrouteinfoall = pd.read_csv('./data/getRouteInfoAll.csv', encoding = 'cp949', index_col = 0)
getrouteinfoall_df = getrouteinfoall[['ROUTE_CD','ROUTE_NO']]

### gestationbyrouteall
getstationbyrouteall = pd.read_csv('./data/getStationByRouteAll.csv', encoding = 'cp949', index_col = 0)
getstationbyrouteall_df1 = getstationbyrouteall[['ROUTE_CD','BUSSTOP_SEQ','BUSSTOP_TP','BUSSTOP_NM','GPS_LATI','GPS_LONG']]
getstationbyrouteall_df2 = getstationbyrouteall[['ROUTE_CD','BUS_STOP_ID','BUSSTOP_SEQ','BUSSTOP_TP','GPS_LATI','GPS_LONG']]

In [2]:
def get_in_off_merged_data_preprocessing(exdata):
    exdata['날짜'] = pd.to_datetime(exdata['날짜'], format='%Y-%m-%d') # '일자' 열을 datetime 형식으로 변환
    exdata['DAY'] = exdata['날짜'].dt.day # 'DAY' 열 생성
    exdata.drop(labels = ['날짜','기종점'], axis = 1, inplace = True) # 사용한 행 제거
    exdata.columns = ['노선', '정류장순번', '정류장', '5','6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23','DAY']
    return exdata

In [3]:
def bus_average_data(exdata, route_no): # 노선별 평균값 데이터 생성 함수
    day_values = [1, 2, 5, 7, 8, 9, 12, 13, 14, 15, 16, 20, 21, 22, 23, 26, 27, 28, 29, 30]

    average_data = pd.DataFrame()

    for day in day_values:
        bus_data = exdata[(exdata['노선'] == route_no) & (exdata['DAY'] == day)].copy() # 특정 노선과 요일에 해당하는 데이터 필터링
        bus_data['ROUTE_NO'] = route_no  # 노선 번호 생성
        bus_data.drop(labels=['노선', '정류장', 'DAY'], axis=1, inplace=True)
        bus_data = bus_data.set_index('정류장순번') # 정류장순번을 index로 설정
        average_data = pd.concat([average_data, bus_data])

    average_data = average_data.groupby('정류장순번').mean() # 선택한 요일들에 대한 각 정류장의 평균값 계산
    average_data['ROUTE_NO'] = average_data['ROUTE_NO'].astype(int) # 정수형으로 변환
    average_data.reset_index(inplace = True)
    return average_data

In [4]:
def get_bus_stop_tp(getstationbyrouteall_df, ROUTE_BUS_NUMBER, ROUTE_NO):
    # 주어진 버스 노선 번호에 해당하는 버스 노선 코드를 얻은 뒤, 노선 코드에 해당하는 버스 정류장 데이터 추출 <순서대로 진행하지 않으면 버스 정류장 데이터가 중복 됨>
    exdata = getstationbyrouteall_df[getstationbyrouteall_df['ROUTE_CD']==ROUTE_BUS_NUMBER[ROUTE_BUS_NUMBER['ROUTE_NO']==ROUTE_NO]['ROUTE_CD'].values[0]].reset_index(drop=True)
    
    return exdata

In [5]:
get_in_off_merged_data_copy = get_in_off_merged_data_preprocessing(get_in_off_merged_data)
get_in_off_merged_data_copy.head()

,노선,정류장순번,정류장,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,DAY
0,102,1,수통골기점지,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,6,0,1
1,102,2,수통골삼거리,1,2,3,6,5,4,4,5,7,9,13,8,8,8,3,5,4,1,0,1
2,102,3,한밭대학교,1,3,6,11,9,9,27,18,16,26,40,45,32,33,9,20,12,16,0,1
3,102,4,삼성화재연수원,0,0,3,9,0,10,30,21,12,33,35,49,34,34,10,26,24,16,0,1
4,102,5,하우스토리/신협연수원,2,5,17,14,12,14,31,22,18,36,40,52,36,37,11,25,26,10,0,1


In [6]:
# 필요한 노선만 필터링한 뒤, int형으로 변환
bus_numbers = ['102', '103', '105', '106', '108', '115', '119', '201', '211', '301', '311', '314', '511', '603', '604', '605', '613', '615', '703', '704', '705', '706', '802']
ROUTE_BUS_NUMBER = getrouteinfoall_df[getrouteinfoall_df['ROUTE_NO'].isin(bus_numbers)]
ROUTE_BUS_NUMBER['ROUTE_NO'] = ROUTE_BUS_NUMBER['ROUTE_NO'].astype(int)

In [7]:
# 각 노선별로 평균 데이터프레임 생성하는 코드 ex) AVERAGE_DATA_102 : 102번 버스의 평균값
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
average_data_dict = {}

for bus_number in bus_numbers:
    average_data = bus_average_data(get_in_off_merged_data_copy, bus_number)
    average_data_dict[bus_number] = average_data

for bus_number in bus_numbers:
    df_name = f"AVERAGE_DATA_{bus_number}"
    globals()[df_name] = average_data_dict[bus_number]

In [8]:
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802] # 위에서 str타입으로 불러왔으니 다시 불러오기

for bus_number in bus_numbers: # 노선(버스)별로 BUSSTOP_TP를 필터링해줌
    variable_name = f"BUS_{bus_number}_BUSSTOP_TP" # BUS_102_BUSSTOP_TP
    globals()[variable_name] = get_bus_stop_tp(getstationbyrouteall_df1, ROUTE_BUS_NUMBER, bus_number)

In [9]:
import pandas as pd

# 버스 노선 번호
bus_numbers = [102, 103, 105, 106, 108, 115, 119, 201, 211, 301, 311, 314, 511, 603, 604, 605, 613, 615, 703, 704, 705, 706, 802]
# r공통
merge_columns = ['ROUTE_NO','BUSSTOP_NM', 'BUSSTOP_TP','5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23','GPS_LATI','GPS_LONG']

merge_results = []

for bus_number in bus_numbers:
    average_data = globals()[f"AVERAGE_DATA_{bus_number}"] 
    busstop_data = globals()[f"BUS_{bus_number}_BUSSTOP_TP"]
    
    merge_result = average_data.merge(busstop_data, left_on='정류장순번', right_on='BUSSTOP_SEQ')
    merge_result = merge_result[merge_columns]  # 필요한 열만 선택
    
    merge_results.append(merge_result)


final_merged_df = pd.concat(merge_results, ignore_index=True)